# Create protein-protein interactome network and assign pathways to proteins

This notebook creates the protein-protein interactome network and checks if proteins in the interactome belong to any pathways in MPath. 

In [1]:
from collections import defaultdict
import getpass
import os
import sys
import time

import pandas as pd

import pyobo

In [2]:
getpass.getuser()

'sarahmubeen'

In [3]:
sys.version

'3.7.8 (v3.7.8:4b47a5b6ba, Jun 27 2020, 04:47:50) \n[Clang 6.0 (clang-600.0.57)]'

In [4]:
time.asctime()

'Mon Jan 18 16:25:51 2021'

In [5]:
# load interaction network
processed_graph = os.path.join('processed_graph_30_04_2020.tsv')
df = pd.read_csv(processed_graph, sep='\t', index_col=0)
df.shape

(253157, 4)

In [6]:
df.head()

,source_identifier,target_identifier,bel_relation,source_database
0,drugbank:DB00087,HGNC:3620,association,drugbank
1,drugbank:DB00818,HGNC:4083,increases,drugbank
2,drugbank:DB00139,HGNC:408,decreases,drugbank
3,drugbank:DB01142,HGNC:278,decreases,drugbank
4,drugbank:DB00173,HGNC:626,association,drugbank


In [7]:
# Contributing resources
df.source_database.value_counts()

BIOGRID           102597
KEGG               63405
PATHWAYCOMMONS     33006
INTACT             23166
CLINICALTRIALS      8157
drugbank            8148
REACTOME            6398
DISGENET            5448
WIKIPATHWAYS        2832
Name: source_database, dtype: int64

In [8]:
def process_interaction_network(df, database_list):
    # Get subset of databases
    protein_protein_df = df.loc[df['source_database'].isin(database_list)]

    # Remove all non-HGNC source and target entities 
    ppi_df = protein_protein_df.loc[
        (protein_protein_df['source_identifier'].str.startswith('HGNC:')) & 
        (protein_protein_df['target_identifier'].str.startswith('HGNC:'))]

    # Remove 'HGNC:' prefix from HGNC identifiers
    network_df = ppi_df.copy()
    network_df['source_identifier'] = network_df['source_identifier'].str.replace('HGNC:', '')
    network_df['target_identifier'] = network_df['target_identifier'].str.replace('HGNC:', '')

    # Remove self edges from network
    network_df.drop(
        network_df[(network_df['source_identifier'] == network_df['target_identifier'])].index, 
        inplace=True
    )
    
    network_df.rename(
        columns={'source_identifier': 'source', 'target_identifier': 'target', 'bel_relation': 'relation'}, 
        inplace=True
    )
        
    return network_df

In [9]:
ppi_databases = ['KEGG','REACTOME','WIKIPATHWAYS','INTACT','BIOGRID','PATHWAYCOMMONS']

network_df = process_interaction_network(df, ppi_databases)
network_df.shape

(203357, 4)

In [10]:
network_df.head()

,source,target,relation,source_database
0,12630,11998,decreases,INTACT
1,10773,11504,association,INTACT
2,8590,8976,association,INTACT
3,13516,6840,association,INTACT
4,903,6773,association,INTACT


### Check if genes in network overlap with genes in MPath gene sets

In [11]:
def spliterate(lines, sep='\t'):
    """Split each line in the iterable by the given separator."""
    for line in lines:
        yield line.strip().split(sep)

def gmt_parser(path):
    """Parse GMT file."""
    with open(path) as file:

        # Get dictionary with pathway and corresponding gene set
        genesets_dict = {
            name: genes
            for name, _, *genes in spliterate(file)
        }
    # Apply gene set filter
    genesets_filter = {
        key: genes
        for key, genes in genesets_dict.items()
    }
    
    return genesets_filter


In [12]:
# Parse MPath gmt file to get MPath gene sets

mpath_geneset_dict = gmt_parser('mpath.txt')

In [13]:
def convert_hgnc_symbols_to_ids(hgnc_mappings, geneset_dict):
    hgnc_id_mappings = {}

    # Convert HGNC symbols in gene sets to HGNC IDs
    for pathway, geneset in geneset_dict.items():
        geneset_ids = []
        for gene in geneset:
            if gene in hgnc_mappings:
                geneset_ids.append(hgnc_mappings[gene])

        hgnc_id_mappings[pathway] = geneset_ids

    return hgnc_id_mappings


In [14]:
# Get HGNC symbols and IDs
hgnc_mappings = pyobo.get_name_id_mapping('hgnc') 

mpath_hgnc_id_mappings = convert_hgnc_symbols_to_ids(hgnc_mappings, mpath_geneset_dict)

print(f'{len(hgnc_mappings)} - Number of all HGNC IDs')
print(f'{len(mpath_hgnc_id_mappings)} - Number of pathways in MPath')

42368 - Number of all HGNC IDs
3068 - Number of pathways in MPath


In [15]:
mpath_hgnc_id_mappings_set = set()

# Get a set of all genes in MPath
for pathway, geneset in mpath_geneset_dict.items():
    for gene in geneset:
        if gene in hgnc_mappings:
            mpath_hgnc_id_mappings_set.add(hgnc_mappings[gene])
            
# Get all genes in the network
network_genes = set(network_df['source'].tolist() + network_df['target'].tolist())

# Get genes in network with membership or no membership to pathways in MPath
non_match = set()
match_genes = set()

for gene in network_genes:
    if gene in mpath_hgnc_id_mappings_set:
        match_genes.add(gene)
    else:
        non_match.add(gene)

print(f'{len(mpath_hgnc_id_mappings_set)} - Number of genes in MPath')
print(f'{len(network_genes)} - Number of genes in protein-protein interactome network')
print(f'{len(match_genes)} - Number of genes in interactome belonging to a pathway in MPath')
print(f'{len(non_match)} - Number of genes in interactome not belonging to a pathway in MPath')


12988 - Number of genes in MPath
8804 - Number of genes in protein-protein interactome network
8603 - Number of genes in interactome belonging to a pathway in MPath
201 - Number of genes in interactome not belonging to a pathway in MPath


In [16]:
# Drop genes not belonging to any MPath pathway from the network
network_df = network_df[~network_df['source'].isin(non_match)]
network_df = network_df[~network_df['target'].isin(non_match)]

network_df.shape

(199535, 4)

In [17]:
network_df.source_database.value_counts()

BIOGRID           97733
KEGG              46983
PATHWAYCOMMONS    31207
INTACT            21124
WIKIPATHWAYS       1826
REACTOME            662
Name: source_database, dtype: int64

In [18]:
network_df.relation.value_counts()

association     144961
increases        21877
regulates        19612
decreases         7296
hasComponent      5789
Name: relation, dtype: int64

In [19]:
network_df.to_csv('interactome_18_01_2021.tsv', sep='\t', index=False)

### Assign pathways to each gene in the network

In [20]:
# Get pathway(s) for genes in network
gene_pathway_dict = defaultdict(list)

for pathway, geneset in mpath_hgnc_id_mappings.items(): 
    for gene in match_genes:
        if gene in geneset:
            gene_pathway_dict[gene].append(pathway)
            
len(gene_pathway_dict)

8603